In [ ]:
# importing the necessary libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist


In [ ]:
# mnist data comes pre-build in tensorflow, we are loading
# the data and splitting it into test and train
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_train.shape)
# By printing the shape of our dataset we can find that 
# X train contains 60000 images of 28x28 pixel and y_train
# contains 60000 labels. Similarly, X_test and y_test contains
# the image and labels respectively

In [ ]:
# In this cell we are reshaping the data to fit the 
# input of neural networkd
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

In [ ]:
# In this column we do one-hot encoding which means, if our label is 3
# then corresponding label will be [0,0,0,1,0,0,0,0,0,0]
# for 4 the label will be [0,0,0,0,4,0,0,0]

from tensorflow.keras.utils import to_categorical
# one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train[0]) #visulaizing the first element of y_train after one-hot encoding

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

model = Sequential()

In [ ]:
# Adding the first convolution layer
model.add(Conv2D(filters=70,kernel_size=3,activation='relu',input_shape=(28,28,1)))
# Here the input shape is 28,28 since the MNIST dataset is 28*28 in size

In [ ]:
# Second convolution layer
model.add(Conv2D(filters=64,kernel_size=3,activation='relu'))

In [ ]:
# After convolving two times we flatten our input such that it is 
# vector with size 1x784 (28x28)
model.add(Flatten())

In [ ]:
# In this cell we use softmax as our activation function
model.add(Dense(10,activation='softmax'))

In [ ]:
model.compile(optimizer='adagrad',metrics=['accuracy'],loss='categorical_crossentropy')

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

In [ ]:
model.compile(optimizer='rmsprop',metrics=['accuracy'],loss='categorical_crossentropy')

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

In [ ]:
model.save('number_predictor.h5')

In [ ]:
model.summary()

In [ ]:
# Optional
# converting tensorlfow model into tflite model
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_model = tf_lite_converter.convert()

In [ ]:
tflite_model_name = 'number_predictor.tflite'
open(tflite_model_name,"wb").write(tf_lite_model)

In [ ]:
# Interpreting our tflite model

interpreter = tf.lite.Interpreter(model_path='number_predictor.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_input_details()
print("input shape: ",input_details[0]['shape'])
print("Input type: ",input_details[0]['dtype'])
print("Output shape: ",output_details[0]['shape'])
print("output type: ",output_details[0]['dtype'])

In [ ]:
image = X_train[82]
plt.imshow(np.squeeze(image),cmap='gray')
plt.show()

In [ ]:
image.shape

In [ ]:
image = image.reshape(1,image.shape[0],image.shape[1],image.shape[2])


In [ ]:
print(image.shape)
pred = model.predict([image])
print('Predicted number is {}'.format(np.argmax(pred)))

# Inception v3 

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt



model = ResNet50(weights='imagenet')

img_path = '/home/vignesh/Desktop/photos/IMG_20191103_193037.jpg'
img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
model.summary()

In [ ]:
import numpy as np

In [ ]:
a = np.array([[1,2],[1,5],[1,3]])

b = np.expand_dims(a,axis=0)

print(a)
print('----------')
print(b)